# കാർട്ട്പോൾ ബാലൻസിംഗ് ചെയ്യാൻ RL പരിശീലനം

ഈ നോട്ട്‌ബുക്ക് [AI for Beginners Curriculum](http://aka.ms/ai-beginners) എന്ന കോഴ്സിന്റെ ഭാഗമാണ്. ഇത് [അധികൃത PyTorch ട്യൂട്ടോറിയൽ](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)നും [ഈ കാർട്ട്പോൾ PyTorch നടപ്പാക്കലിനും](https://github.com/yc930401/Actor-Critic-pytorch) പ്രചോദനം നൽകിയതാണ്.

ഈ ഉദാഹരണത്തിൽ, ഒരു മോഡൽ ഒരു കാർട്ടിൽ മുകളിലായി നില്ക്കുന്ന പോളിനെ ഇടത്തും വലത്തും നീങ്ങുന്ന ഹോരിസോണ്ടൽ സ്കെയിലിൽ ബാലൻസ് ചെയ്യാൻ RL ഉപയോഗിച്ച് പരിശീലിപ്പിക്കും. പോളിനെ സിമുലേറ്റ് ചെയ്യാൻ [OpenAI Gym](https://www.gymlibrary.ml/) പരിസ്ഥിതി ഉപയോഗിക്കും.

> **Note**: ഈ പാഠത്തിന്റെ കോഡ് നിങ്ങൾക്ക് ലോക്കലായി (ഉദാ. Visual Studio Code-ൽ) ഓടിക്കാം, അപ്പോൾ സിമുലേഷൻ പുതിയ വിൻഡോയിൽ തുറക്കും. ഓൺലൈനിൽ കോഡ് ഓടിക്കുമ്പോൾ, [ഇവിടെ](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) വിവരിച്ചിരിക്കുന്നതുപോലെ ചില മാറ്റങ്ങൾ വരുത്തേണ്ടി വരാം.

ആദ്യം Gym ഇൻസ്റ്റാൾ ചെയ്തിട്ടുണ്ടെന്ന് ഉറപ്പാക്കാം:


In [ ]:
import sys
!{sys.executable} -m pip install gym

ഇപ്പോൾ നമുക്ക് CartPole പരിസ്ഥിതി സൃഷ്ടിച്ച് അതിൽ എങ്ങനെ പ്രവർത്തിക്കാമെന്ന് നോക്കാം. ഒരു പരിസ്ഥിതിക്ക് താഴെപ്പറയുന്ന ഗുണങ്ങൾ ഉണ്ടാകും:

* **Action space** എന്നത് സിമുലേഷന്റെ ഓരോ ഘട്ടത്തിലും നാം ചെയ്യാൻ കഴിയുന്ന പ്രവർത്തനങ്ങളുടെ സമാഹാരമാണ്
* **Observation space** എന്നത് നാം നടത്താൻ കഴിയുന്ന നിരീക്ഷണങ്ങളുടെ സ്ഥലം ആണ്


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

സിമുലേഷൻ എങ്ങനെ പ്രവർത്തിക്കുന്നു എന്ന് നോക്കാം. താഴെയുള്ള ലൂപ്പ് സിമുലേഷൻ നടത്തുന്നു, `env.step` അവസാനിപ്പിക്കൽ ഫ്ലാഗ് `done` തിരികെ നൽകുന്നത് വരെ. നാം ക്രമരഹിതമായി പ്രവർത്തനങ്ങൾ തിരഞ്ഞെടുക്കും `env.action_space.sample()` ഉപയോഗിച്ച്, അതായത് പരീക്ഷണം വളരെ വേഗം പരാജയപ്പെടാൻ സാധ്യതയുണ്ട് (CartPole പരിസ്ഥിതി CartPole-യുടെ വേഗത, സ്ഥാനം അല്ലെങ്കിൽ കോണം ചില പരിധികൾക്കു പുറത്തായാൽ അവസാനിക്കും).

> സിമുലേഷൻ പുതിയ വിൻഡോയിൽ തുറക്കും. നിങ്ങൾക്ക് കോഡ് പലതവണ ഓടിച്ച് അത് എങ്ങനെ പ്രവർത്തിക്കുന്നു എന്ന് കാണാം.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

നിങ്ങൾ ശ്രദ്ധിക്കാം, നിരീക്ഷണങ്ങളിൽ 4 സംഖ്യകൾ അടങ്ങിയിരിക്കുന്നു. അവയാണ്:
- കാർട്ടിന്റെ സ്ഥാനം
- കാർട്ടിന്റെ വേഗത
- പോളിന്റെ കോണം
- പോളിന്റെ തിരിവിന്റെ വേഗത

`rew` എന്നത് ഓരോ ഘട്ടത്തിലും ലഭിക്കുന്ന റിവാർഡാണ്. CartPole പരിസ്ഥിതിയിൽ ഓരോ സിമുലേഷൻ ഘട്ടത്തിനും നിങ്ങൾക്ക് 1 പോയിന്റ് ലഭിക്കുന്നുവെന്ന് കാണാം, ലക്ഷ്യം ആകെ റിവാർഡ് പരമാവധി ആക്കുകയാണ്, അതായത് CartPole വീഴാതെ തുല്യസ്ഥിതിയിൽ നിലനിൽക്കാൻ കഴിയുന്ന സമയം.

റീൻഫോഴ്‌സ്‌മെന്റ് ലേണിങ്ങിന്റെ സമയത്ത്, നമ്മുടെ ലക്ഷ്യം ഒരു **പോളിസി** $\pi$ പരിശീലിപ്പിക്കുകയാണ്, അത് ഓരോ സ്റ്റേറ്റ് $s$-ക്കും എടുക്കേണ്ട ആക്ഷൻ $a$ പറയുന്നവയാണ്, അതായത് $a = \pi(s)$.

നിങ്ങൾ പ്രൊബബിലിസ്റ്റിക് പരിഹാരമുണ്ടാക്കാൻ ആഗ്രഹിക്കുന്നുവെങ്കിൽ, ഓരോ ആക്ഷനിനും ഒരു പ്രൊബബിലിറ്റി സെറ്റ് നൽകുന്ന രീതിയിലാണ് പോളിസിയെ കാണുന്നത്, അതായത് $\pi(a|s)$ എന്നത് സ്റ്റേറ്റ് $s$-ൽ ആക്ഷൻ $a$ എടുക്കേണ്ടതിന്റെ സാധ്യതയെ സൂചിപ്പിക്കും.

## പോളിസി ഗ്രേഡിയന്റ് മെത്തഡ്

ഏറ്റവും ലളിതമായ RL ആൽഗോരിതമായ **പോളിസി ഗ്രേഡിയന്റ്**-ൽ, അടുത്ത ആക്ഷൻ പ്രവചിക്കാൻ ഒരു ന്യൂറൽ നെറ്റ്‌വർക്ക് പരിശീലിപ്പിക്കും.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

നാം നിരവധി പരീക്ഷണങ്ങൾ നടത്തിക്കൊണ്ട് നെറ്റ്‌വർക്ക് പരിശീലിപ്പിക്കും, ഓരോ റൺ കഴിഞ്ഞ് നമ്മുടെ നെറ്റ്‌വർക്ക് അപ്ഡേറ്റ് ചെയ്യും. പരീക്ഷണം നടത്തുകയും ഫലങ്ങൾ (അഥവാ **ട്രേസ്**) തിരികെ നൽകുകയും ചെയ്യുന്ന ഒരു ഫംഗ്ഷൻ നിർവചിക്കാം - എല്ലാ സ്റ്റേറ്റുകളും, ആക്ഷനുകളും (അവയുടെ ശുപാർശ ചെയ്ത സാധ്യതകളും), റിവാർഡുകളും:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

നിങ്ങൾ ഒരു എപ്പിസോഡ് പരിശീലനമില്ലാത്ത നെറ്റ്‌വർക്കുമായി ഓടിച്ച് ആകെ റിവാർഡ് (അഥവാ എപ്പിസോഡിന്റെ ദൈർഘ്യം) വളരെ കുറവാണെന്ന് കാണാം:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

പോളിസി ഗ്രേഡിയന്റ് ആൽഗോറിതത്തിന്റെ ഒരു സങ്കീർണ്ണമായ ഭാഗം **ഡിസ്‌കൗണ്ടഡ് റിവാർഡുകൾ** ഉപയോഗിക്കുകയാണ്. കളിയുടെ ഓരോ ഘട്ടത്തിലും മൊത്തം റിവാർഡുകളുടെ വെക്ടർ കണക്കാക്കുകയാണ് ആശയം, ഈ പ്രക്രിയയിൽ ആദ്യം ലഭിക്കുന്ന റിവാർഡുകൾ ഒരു കോഫിഷ്യന്റ് $gamma$ ഉപയോഗിച്ച് ഡിസ്‌കൗണ്ട് ചെയ്യുന്നു. നമുക്ക് ഇത് പരിശീലനത്തെ ബാധിക്കുന്ന ഭാരമായി ഉപയോഗിക്കേണ്ടതിനാൽ, ലഭിച്ച വെക്ടർ നോർമലൈസ് ചെയ്യുകയും ചെയ്യുന്നു:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

ഇപ്പോൾ നമുക്ക് യഥാർത്ഥ പരിശീലനം തുടങ്ങാം! നാം 300 എപ്പിസോഡുകൾ നടത്തും, ഓരോ എപ്പിസോഡിലും നാം താഴെ പറയുന്ന കാര്യങ്ങൾ ചെയ്യും:

1. പരീക്ഷണം നടത്തുകയും ട്രേസ് ശേഖരിക്കുകയും ചെയ്യുക  
2. എടുത്ത പ്രവർത്തനങ്ങളും പ്രവചിച്ച സാധ്യതകളും തമ്മിലുള്ള വ്യത്യാസം (`gradients`) കണക്കാക്കുക. വ്യത്യാസം കുറവായിരിക്കും എങ്കിൽ, നാം ശരിയായ പ്രവർത്തനം എടുത്തതായി കൂടുതൽ ഉറപ്പുണ്ടാകും.  
3. ഡിസ്കൗണ്ടഡ് റിവാർഡുകൾ കണക്കാക്കി, ഗ്രാഡിയന്റുകൾക്ക് ഡിസ്കൗണ്ടഡ് റിവാർഡുകൾ ഗുണിക്കുക - ഇതിലൂടെ ഉയർന്ന റിവാർഡ് ലഭിച്ച ഘട്ടങ്ങൾ താഴ്ന്ന റിവാർഡ് ലഭിച്ച ഘട്ടങ്ങളെക്കാൾ അന്തിമ ഫലത്തിൽ കൂടുതൽ സ്വാധീനം ചെലുത്തും.  
4. നമ്മുടെ ന്യൂറൽ നെറ്റ്‌വർക്കിന്റെ പ്രതീക്ഷിച്ച ലക്ഷ്യ പ്രവർത്തനങ്ങൾ partly റൺ സമയത്ത് പ്രവചിച്ച സാധ്യതകളിൽ നിന്നും partly കണക്കാക്കിയ ഗ്രാഡിയന്റുകളിൽ നിന്നുമാകും. ഗ്രാഡിയന്റുകളും റിവാർഡുകളും എത്രമാത്രം പരിഗണിക്കണമെന്ന് നിശ്ചയിക്കാൻ `alpha` പാരാമീറ്റർ ഉപയോഗിക്കും - ഇതാണ് റീ ഇൻഫോഴ്‌സ്‌മെന്റ് ആൽഗോരിതത്തിന്റെ *learning rate*.  
5. അവസാനം, നാം സ്റ്റേറ്റുകളും പ്രതീക്ഷിച്ച പ്രവർത്തനങ്ങളും ഉപയോഗിച്ച് നെറ്റ്‌വർക്ക് പരിശീലിപ്പിച്ച്, ഈ പ്രക്രിയ ആവർത്തിക്കും.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

ഇപ്പോൾ ഫലങ്ങൾ കാണാൻ റെൻഡറിംഗ് സഹിതം എപ്പിസോഡ് ഓടിക്കാം:


In [ ]:
_ = run_episode(render=True)

നമ്മൾ ഇപ്പോൾ കാണാൻ കഴിയുന്നത് പോൾ വളരെ നന്നായി ബാലൻസ് ചെയ്യാൻ കഴിഞ്ഞു എന്നതാണ്!

## ആക്ടർ-ക്രിട്ടിക് മോഡൽ

ആക്ടർ-ക്രിട്ടിക് മോഡൽ പോളിസി ഗ്രേഡിയന്റുകളുടെ കൂടുതൽ വികസനമാണ്, ഇതിൽ നാം ഒരു ന്യൂറൽ നെറ്റ്‌വർക്ക് നിർമ്മിച്ച് പോളിസിയും അളവിട്ട റിവാർഡുകളും പഠിപ്പിക്കുന്നു. നെറ്റ്‌വർക്ക് രണ്ട് ഔട്ട്പുട്ടുകൾ ഉണ്ടാകും (അല്ലെങ്കിൽ ഇത് രണ്ട് വേർതിരിച്ച നെറ്റ്‌വർക്കുകളായി കാണാം):
* **ആക്ടർ** പോളിസി ഗ്രേഡിയന്റ് മോഡലിൽപോലെ സ്റ്റേറ്റ് പ്രൊബബിലിറ്റി ഡിസ്‌ട്രിബ്യൂഷൻ നൽകിയാണ് എടുക്കേണ്ട ആക്ഷൻ നിർദ്ദേശിക്കുന്നത്
* **ക്രിട്ടിക്** ആ ആക്ഷനുകളിൽ നിന്നുണ്ടാകുന്ന റിവാർഡ് എത്രയാകും എന്ന് അളക്കുന്നു. നൽകിയ സ്റ്റേറ്റിൽ ഭാവിയിൽ ലഭിക്കാവുന്ന മൊത്തം അളവിട്ട റിവാർഡുകൾ ഇത് തിരിച്ചുകൊടുക്കും.

ഇത്തരമൊരു മോഡൽ നിർവചിക്കാം:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

നമുക്ക് നമ്മുടെ `discounted_rewards` மற்றும் `run_episode` ഫംഗ്ഷനുകൾ ചെറിയ മാറ്റം വരുത്തേണ്ടതുണ്ട്:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


ഇപ്പോൾ നാം പ്രധാന പരിശീലന ലൂപ്പ് നടത്തും. ശരിയായ നഷ്ട ഫംഗ്ഷനുകൾ കണക്കാക്കി നെറ്റ്‌വർക്ക് പാരാമീറ്ററുകൾ അപ്ഡേറ്റ് ചെയ്ത് മാനുവൽ നെറ്റ്‌വർക്ക് പരിശീലന പ്രക്രിയ ഉപയോഗിക്കും:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

അവസാനമായി, നമുക്ക് പരിസ്ഥിതിയെ അടയ്ക്കാം.


In [ ]:
env.close()

## പ്രധാനപ്പെട്ട കാര്യങ്ങൾ

ഈ ഡെമോയിൽ നാം രണ്ട് RL ആൽഗോരിതങ്ങൾ കണ്ടു: ലളിതമായ പോളിസി ഗ്രേഡിയന്റ്, കൂടാതെ കൂടുതൽ സങ്കീർണ്ണമായ ആക്ടർ-ക്രിട്ടിക്. ആൽഗോരിതങ്ങൾ സ്റ്റേറ്റ്, ആക്ഷൻ, റിവാർഡ് എന്ന аб്സ്ട്രാക്റ്റ് ആശയങ്ങളുമായി പ്രവർത്തിക്കുന്നതായതിനാൽ, അവ വളരെ വ്യത്യസ്തമായ പരിസ്ഥിതികളിൽ പ്രയോഗിക്കാവുന്നതാണ്.

റീ ഇൻഫോഴ്‌സ്‌മെന്റ് ലേണിംഗ് നമ്മെ പ്രശ്നം പരിഹരിക്കാൻ ഏറ്റവും നല്ല തന്ത്രം പഠിക്കാനാകും, അവസാന റിവാർഡ് നോക്കിയാൽ മതി. ലേബൽ ചെയ്ത ഡാറ്റാസെറ്റുകൾ ആവശ്യമില്ലാത്തതുകൊണ്ട്, നമ്മുടെ മോഡലുകൾ മെച്ചപ്പെടുത്താൻ സിമുലേഷനുകൾ പലതവണ ആവർത്തിക്കാം. എങ്കിലും, RL-യിൽ ഇപ്പോഴും നിരവധി വെല്ലുവിളികൾ ഉണ്ട്, ഈ രസകരമായ AI മേഖലയിലേക്ക് കൂടുതൽ ശ്രദ്ധ കേന്ദ്രീകരിക്കാൻ നിങ്ങൾ തീരുമാനിച്ചാൽ അവയെക്കുറിച്ച് പഠിക്കാം.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖ അധികാരപരമായ ഉറവിടമായി കണക്കാക്കണം. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനത്തിന്റെ ഉപയോഗത്തിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
